In [1]:
from numpy import *
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import cv2
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
df = pd.read_csv("../input/severstal-steel-defect-detection/train.csv")
df.head()

In [4]:
df.shape

In [5]:
df.ClassId.value_counts()
plt.figure(figsize=(10,5))
df.ClassId.value_counts().plot(kind='bar')

In [6]:
i = []
l = []
for img,classid,ep in tqdm(df.values):
    image=cv2.imread("/kaggle/input/severstal-steel-defect-detection/train_images/{}".format(img),cv2.IMREAD_COLOR)
    image=cv2.resize(image,(120,120))
    i.append(image)
    l.append(classid)

In [7]:
print(i[15])

In [8]:
plt.imshow(i[15])

In [11]:
plt.imshow(i[25])

In [15]:
from sklearn.preprocessing import LabelEncoder
en = LabelEncoder()

X = array(i)
X = X/255

y = en.fit_transform(l)
y = tf.keras.utils.to_categorical(y)

print(y)

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),input_shape=(120,120,3),activation='relu'),
    tf.keras.layers.MaxPool2D(3,3),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(3,3),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(3,3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4,activation='sigmoid')
])

In [18]:
model.summary()

In [19]:
es = tf.keras.callbacks.EarlyStopping(patience=5,min_delta=0.001,restore_best_weights=True)

In [20]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,stratify=y,shuffle=True)

In [22]:
model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test),batch_size=128,verbose=1,callbacks=[es])

In [26]:
hist = pd.DataFrame(model.history.history)
hist.loc[:, ['loss','val_loss']].plot(title='Loss Curve')

In [27]:
hist.loc[:, ['acc','val_acc']].plot(title='Accuracy Curve')

In [33]:
plt.imshow(X_train[15])
p = X_train[15]
yp = expand_dims(p, axis=0)
print("Falls under",argmax(model.predict(yp)))